In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np

In [7]:
# Função para redimensionar imagens para baixa resolução
def create_low_resolution_image(image, scale=0.25):
    return image.resize((int(image.size[0] * scale), int(image.size[1] * scale)), Image.BICUBIC)

# Função para carregar imagens do diretório e criar pares HR-LR
def load_images_from_directory(directory, target_size):
    images_hr = []
    images_lr = []
    
    # Listar todos os arquivos no diretório
    files = os.listdir(directory)
    print(f"Arquivos encontrados no diretório {directory}: {len(files)} arquivos")

    for filename in files:
        if filename.endswith(".png") or filename.endswith(".jpg"):  # Suporte para .png e .jpg
            file_path = os.path.join(directory, filename)
            print(f"Carregando imagem: {file_path}")
            try:
                img = Image.open(file_path).convert('RGB')  # Carregar a imagem
                img_hr = img.resize(target_size, Image.BICUBIC)
                img_lr = create_low_resolution_image(img_hr)
                
                # Converter as imagens em arrays numpy
                img_hr = np.array(img_hr)
                img_lr = np.array(img_lr)
                
                # Adicionar imagens HR e LR às listas
                images_hr.append(img_hr)
                images_lr.append(img_lr)

            except Exception as e:
                print(f"Erro ao carregar a imagem {filename}: {e}")
    
    print(f"Total de imagens HR carregadas: {len(images_hr)}")
    print(f"Total de imagens LR carregadas: {len(images_lr)}")
    
    return np.array(images_hr), np.array(images_lr)


In [8]:
# Carregar imagens de alta e baixa resolução
div2k_train_hr_dir = "C:\\Users\\anton\\OneDrive\\Documentos\\ML_Deep\\Div2K\\DIV2K_train_HR\\DIV2K_train_HR"
div2k_val_hr_dir = "C:\\Users\\anton\\OneDrive\\Documentos\\ML_Deep\\Div2K\\DIV2K_valid_HR\\DIV2K_valid_HR"

In [9]:
import os

# Verificar se o diretório contém as imagens
files = os.listdir(div2k_train_hr_dir)
print(f"Arquivos encontrados: {files}")

Arquivos encontrados: ['0001.png', '0002.png', '0003.png', '0004.png', '0005.png', '0006.png', '0007.png', '0008.png', '0009.png', '0010.png', '0011.png', '0012.png', '0013.png', '0014.png', '0015.png', '0016.png', '0017.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0023.png', '0024.png', '0025.png', '0026.png', '0027.png', '0028.png', '0029.png', '0030.png', '0031.png', '0032.png', '0033.png', '0034.png', '0035.png', '0036.png', '0037.png', '0038.png', '0039.png', '0040.png', '0041.png', '0042.png', '0043.png', '0044.png', '0045.png', '0046.png', '0047.png', '0048.png', '0049.png', '0050.png', '0051.png', '0052.png', '0053.png', '0054.png', '0055.png', '0056.png', '0057.png', '0058.png', '0059.png', '0060.png', '0061.png', '0062.png', '0063.png', '0064.png', '0065.png', '0066.png', '0067.png', '0068.png', '0069.png', '0070.png', '0071.png', '0072.png', '0073.png', '0074.png', '0075.png', '0076.png', '0077.png', '0078.png', '0079.png', '0080.png', '0081.png', '0082

In [10]:
# Definir o tamanho das imagens de alta resolução para reescalonar
target_size = (128, 128)

train_hr, train_lr = load_images_from_directory(div2k_train_hr_dir, target_size)
val_hr, val_lr = load_images_from_directory(div2k_val_hr_dir, target_size)

# Redimensionar as imagens de treinamento e validação
train_lr_resized = tf.image.resize(train_lr, [128, 128])
val_lr_resized = tf.image.resize(val_lr, [128, 128])

# Normalizar as imagens para o intervalo [0, 1]
train_hr = train_hr / 255.0 if len(train_hr) > 0 else train_hr
train_lr = train_lr / 255.0 if len(train_lr) > 0 else train_lr
val_hr = val_hr / 255.0 if len(val_hr) > 0 else val_hr
val_lr = val_lr / 255.0 if len(val_lr) > 0 else val_lr

# Checar as dimensões para garantir que estão corretas
print("Dimensões de train_hr:", train_hr.shape)
print("Dimensões de train_lr:", train_lr.shape)
print("Dimensões de val_hr:", val_hr.shape)
print("Dimensões de val_lr:", val_lr.shape)

Arquivos encontrados no diretório C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR: 800 arquivos
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0001.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0002.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0003.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0004.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0005.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0006.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0007.png
Carregando imagem: C:\Users\anton\OneDrive\Documentos\ML_Deep\Div2K\DIV2K_train_HR\DIV2K_train_HR\0008.png
Carregando imagem

In [11]:
from tensorflow.keras import layers, models

# Função auxiliar para bloco residual com normalização
def residual_block(input_layer, filters):
    # Convolução principal
    x = layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(input_layer)
    x = layers.BatchNormalization()(x)  # Normalização em lote
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)  # Normalização em lote

    # Ajuste do número de canais da entrada para coincidir com os filtros
    if input_layer.shape[-1] != filters:
        input_layer = layers.Conv2D(filters, (1, 1), padding='same')(input_layer)

    # Skip connection (bloco residual)
    x = layers.add([x, input_layer])
    x = layers.Activation('relu')(x)
    return x

# Função para construir a arquitetura da U-Net com melhorias
def unet_autoencoder(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    conv1 = residual_block(inputs, 64)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = residual_block(pool1, 128)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = residual_block(pool2, 256)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = residual_block(pool3, 512)

    # Decoder
    up1 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv4)
    merge1 = layers.concatenate([up1, conv3], axis=3)
    conv5 = residual_block(merge1, 256)

    up2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge2 = layers.concatenate([up2, conv2], axis=3)
    conv6 = residual_block(merge2, 128)

    up3 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge3 = layers.concatenate([up3, conv1], axis=3)
    conv7 = residual_block(merge3, 64)

    # Camada de saída
    outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(conv7)

    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model



In [12]:
# Parâmetros
input_shape = (128, 128, 3)
autoencoder = unet_autoencoder(input_shape)
autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss=MeanSquaredError(), metrics=['accuracy'])


In [ ]:
history = autoencoder.fit(
    train_lr_resized, train_hr,  # Imagens de baixa resolução redimensionadas como entrada, alta resolução como saída
    epochs=100,
    batch_size=32,
    validation_data=(val_lr_resized, val_hr)
)


In [ ]:
# Avaliando o modelo
autoencoder.evaluate(val_lr, val_hr)

In [ ]:
import tensorflow as tf

# Redimensionar as imagens de baixa resolução para o tamanho esperado
val_lr_resized = tf.image.resize(val_lr, [128, 128])

# Avaliar o modelo com as imagens de validação redimensionadas
autoencoder.evaluate(val_lr_resized, val_hr)


In [ ]:
# Salvando o modelo
autoencoder.save('autoencoder_unet2_div2k.h5')